In [12]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"

In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM


tokenizer = AutoTokenizer.from_pretrained('distilgpt2')
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained('distilgpt2')

In [14]:
#data laoding and formatting
from datasets import load_dataset
dataset = load_dataset("json", data_files=r"C:\Users\Admin\Downloads\mini_email_dataset.json")

In [16]:
#inspecting sample from the loaded dataset
print(dataset)
print(dataset['train'][0])

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 20
    })
})
{'instruction': 'Convert the input into a crisp professional email with a subject line.', 'input': 'Tell Derrick there are still surrendered items in the report. Ask him to check again.', 'output': 'Subject: Review of Surrendered Items\n\nHi Derrick,\n\nThere are still surrendered items showing in the report. Please go through it again and ensure everything is properly accounted for.\n\nBest regards,\nBrian'}


In [17]:
#formatting
def format(example):
    prompt = f"### Instruction:\n{example['instruction']}\n\n### Input:{example['input']}\n\n### Response:\n{example['output']}"
    return{'text' : prompt}

In [18]:
#applying the formatting to the entire dataset
formatted_dataset = dataset['train'].map(format)

In [20]:
print(formatted_dataset)

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 20
})


In [21]:
#defining tokenization function
def tokenize_function(example):
    return tokenizer(
        example['text'],
        padding = 'max_length',
        truncation = True,
        max_length = 512,
        return_tensors = 'pt'
    )

In [22]:
#appying the tokenizer
tokenized_dataset = formatted_dataset.map(tokenize_function, batched = True)

In [23]:
print(tokenized_dataset[0])

{'instruction': 'Convert the input into a crisp professional email with a subject line.', 'input': 'Tell Derrick there are still surrendered items in the report. Ask him to check again.', 'output': 'Subject: Review of Surrendered Items\n\nHi Derrick,\n\nThere are still surrendered items showing in the report. Please go through it again and ensure everything is properly accounted for.\n\nBest regards,\nBrian', 'text': '### Instruction:\nConvert the input into a crisp professional email with a subject line.\n\n### Input:Tell Derrick there are still surrendered items in the report. Ask him to check again.\n\n### Response:\nSubject: Review of Surrendered Items\n\nHi Derrick,\n\nThere are still surrendered items showing in the report. Please go through it again and ensure everything is properly accounted for.\n\nBest regards,\nBrian', 'input_ids': [21017, 46486, 25, 198, 3103, 1851, 262, 5128, 656, 257, 23453, 4708, 3053, 351, 257, 2426, 1627, 13, 198, 198, 21017, 23412, 25, 24446, 32979, 6

In [25]:
pip install -U "accelerate>=0.26.0"

Note: you may need to restart the kernel to use updated packages.


In [26]:
#fine config
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir = "./distilgpt2-email-finetuned",
    per_device_train_batch_size = 1,
    num_train_epochs = 4,
    logging_steps = 10,
    save_steps = 10,
    save_total_limit=2,                
    fp16=False
)

RuntimeError: Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_tf_utils because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.